In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from sparkmeasure import StageMetrics
spark=SparkSession \
        .builder \
        .master("spark://ubuntu:7077")\
        .appName("A Boring Query")\
        .getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True)

In [2]:
stagemetrics = StageMetrics(spark)

In [ ]:
stagemetrics.begin()
Movies_Dataframe=(spark.read.format("csv")
            .option("header","True")
            .option("inferSchema","True")
            .load("/home/meli/Documents/Kaggle/movie.csv")
      )
Tags_Dataframe=(spark.read.format("csv")
            .option("header","True")
            .option("inferSchema","True")
            .load("/home/meli/Documents/Kaggle/tag.csv")
      )

In [ ]:
Tags_Dataframe=Tags_Dataframe.withColumn('tag',lower(col('tag'))).withColumn('tag',regexp_replace('tag', '[^\w\s]+', ''))
stagemetrics.end()

In [ ]:
Joined_Dataframe=(Tags_Dataframe.join(Movies_Dataframe,Movies_Dataframe.movieId==Tags_Dataframe.movieId)
                  .select(Movies_Dataframe.title,Tags_Dataframe.userId,Tags_Dataframe.movieId,Tags_Dataframe.tag)
                  .filter("tag== 'boring' ")
                  .dropDuplicates(subset=['movieId'])
                 )
Joined_Dataframe.select("title").sort('title').show(truncate=0)

In [ ]:
stagemetrics.print_report()